In [1]:
import cv2
import csv
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import torch

from typing import List, Dict
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset

In [3]:
# load labels
img_filenames = []
with open("train.csv", 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
    _ = next(csvreader)
    for i in csvreader:
        img_filenames.append(i)
        
classes = []
with open("category.csv", 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
    _ = next(csvreader)
    for i in csvreader:
        classes.append(i[1])

print("total number of images:", len(img_filenames))
print("total number of classes:", len(classes))

total number of images: 69540
total number of classes: 100


In [14]:
# find faces and crop
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
data_path = "C:\\Users\\USER\\Downloads\\mini\\train\\train"
img_path = "C:\\Users\\USER\\Downloads\\mini\\train\\train_crop\\"

for i in range(len(img_filenames)):
    # Read the input image
    path = data_path + "\\" + img_filenames[i][1]
    img = cv2.imread(path)
    
    if img is None or img.size == 0:
        continue
    
    # if not 1~256
    if img.any() > 1:
        _
    else:
        img = img * 256
    img = img.astype(np.uint8)
    
    # grayscale images
    if len(img.shape) == 2:
        print("gray", i)
        gray = img
        
    # RGBA images
    elif img.shape[2] == 4:
        print("RGBA", i)
        re_img = img[:, :, :3]
        cv_img = cv2.UMat(re_img)
        gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
        
    # RGB images
    else:
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
              
    faces = face_cascade.detectMultiScale(gray, 1.1, 9)
    
    if len(faces) > 0:
        max_img = faces[0]
        maxwh = 0
        for (x, y, w, h) in faces:
            if w*h > maxwh:
                max_img = img[y:y+h, x:x+w]
                imgfilename = img_path + str(i) + ".jpg"
        cv2.imwrite(imgfilename, max_img)
        
print("Crop Done.")

Crop Done.


In [15]:
def generate_filename_csv(path, csv_filename):
    files = os.listdir(path)
    img_dataset = []
    
    for i in files:
        filename = i.split(".")
        labelname = img_filenames[int(filename[0])][2]
        if labelname in classes:
            img_label = classes.index(labelname)
#             print(i, labelname, img_label)
            img_dataset.append([i, img_label])
    
    print(len(files), len(img_dataset))
    
    with open(csv_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(img_dataset)
        f.close()

generate_filename_csv("C:\\Users\\USER\\Downloads\\mini\\train\\train_crop", "train_filename.csv")

54484 54484


In [28]:
def crop_test():
    data_path = "C:\\Users\\USER\\Downloads\\mini\\test\\test"
    img_path = "C:\\Users\\USER\\Downloads\\mini\\test\\test_crop\\"
    testfiles = os.listdir(data_path)
    test_filename = []
    for file_name in testfiles:
        test_filename.append(file_name)
    
    # find faces and crop
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    for i in test_filename:
        # Read the input image
        path = data_path + "\\" + i
        img = cv2.imread(path)
        
        if img is None or img.size == 0:
            continue

        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 9)
        
        if len(faces) > 0:
            number = i.split(".")
            max_img = faces[0]
            maxwh = 0
            for (x, y, w, h) in faces:
                if w*h > maxwh:
                    max_img = img[y:y+h, x:x+w]
                    imgfilename = img_path + number[0] + ".jpg"
            cv2.imwrite(imgfilename, max_img)
            print(imgfilename)
    
crop_test()
print("Crop Test Done.")

RGBA 101.jpg
RGBA 1054.jpg
RGBA 107.jpg
RGBA 1073.jpg
RGBA 1087.jpg
RGBA 11.jpg
gray 1120.jpg
gray 1137.jpg
RGBA 1140.jpg
gray 1178.jpg
RGBA 1201.jpg
RGBA 1208.jpg
RGBA 1211.jpg
gray 1214.jpg
RGBA 1231.jpg
gray 1233.jpg
RGBA 1259.jpg
RGBA 1279.jpg
RGBA 1280.jpg
gray 1288.jpg
RGBA 132.jpg
RGBA 1355.jpg
gray 1377.jpg
RGBA 1382.jpg
RGBA 1393.jpg
RGBA 1407.jpg
gray 1509.jpg
RGBA 1514.jpg
RGBA 1544.jpg
RGBA 1550.jpg
gray 1555.jpg
RGBA 1571.jpg
RGBA 1590.jpg
RGBA 1606.jpg
RGBA 1618.jpg
RGBA 1639.jpg
RGBA 165.jpg
RGBA 1651.jpg
RGBA 1700.jpg
RGBA 1722.jpg
gray 1734.jpg
RGBA 1735.jpg
RGBA 1743.jpg
RGBA 1749.jpg
RGBA 1754.jpg
gray 1760.jpg
RGBA 1784.jpg
RGBA 1793.jpg
RGBA 1819.jpg
gray 1833.jpg
gray 1848.jpg
RGBA 1850.jpg
RGBA 1882.jpg
RGBA 1947.jpg
RGBA 1948.jpg
RGBA 1998.jpg
gray 2005.jpg
RGBA 2037.jpg
gray 2044.jpg
RGBA 2095.jpg
RGBA 2151.jpg
RGBA 2166.jpg
gray 2196.jpg
RGBA 2220.jpg
RGBA 2229.jpg
RGBA 224.jpg
RGBA 2324.jpg
RGBA 2337.jpg
RGBA 2343.jpg
RGBA 2365.jpg
gray 2371.jpg
RGBA 238.jpg


In [16]:
files = os.listdir("C:\\Users\\USER\\Downloads\\mini\\test\\test_crop")   
with open("test_filename.txt", 'w', newline='') as f:
    for i in files:
        f.write(i+" ")
    f.close()